In [ ]:
import pandas as pd
import numpy as np 
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tools.sm_exceptions import ValueWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', ValueWarning)
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
from datetime import date, timedelta
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pmdarima import auto_arima
from yellowbrick.regressor import prediction_error
from yellowbrick.regressor import residuals_plot

In [ ]:
countries = ['India', 'USA', 'United Kingdom', 'Russia', 'Israel']

In [ ]:
countries1 = ['India', 'USA', 'Russia', 'Israel']

In [ ]:
CB91_Blue = '#2CBDFE'
CB91_Green = '#47DBCD'
CB91_Pink = '#F3A0F2'
CB91_Purple = '#9D2EC5'
CB91_Violet = '#661D98'
CB91_Amber = '#F5B14C'
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
color_list = [CB91_Blue, CB91_Pink, CB91_Green, CB91_Amber,
              CB91_Purple, CB91_Violet]
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=color_list)

In [ ]:
sns.set(font='Franklin Gothic Book',
        rc={
 'axes.axisbelow': False,
 'axes.edgecolor': 'lightgrey',
 'axes.facecolor': 'None',
 'axes.grid': False,
 'axes.labelcolor': 'dimgrey',
 'axes.spines.right': False,
 'axes.spines.top': False,
 'figure.facecolor': 'white',
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'text.color': 'dimgrey',
 'xtick.bottom': False,
 'xtick.color': 'dimgrey',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': 'dimgrey',
 'ytick.direction': 'out',
 'ytick.left': False,
 'ytick.right': False})
sns.set_context("notebook", rc={"font.size":16,
                                "axes.titlesize":20,
                                "axes.labelsize":18})

In [ ]:
csv_data = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
df = pd.read_csv(csv_data, usecols= ["location", "date", "new_cases", "new_tests", "new_deaths", 'new_vaccinations'])
df['date'] = pd.to_datetime(df.date)
df.dropna(inplace=True)
df.head()
df.isna()

In [ ]:
data_ind = df[df["location"] == "India"] #India
data_ind_features = data_ind.drop(['location', 'date', 'new_cases'], axis = 1)
data_ind_target = data_ind['new_cases'].copy()

data_usa = df[df['location'] == 'United States'] #United States
data_usa_features = data_usa.drop(['location', 'date', 'new_cases'], axis = 1)
data_usa_target = data_usa['new_cases'].copy()

data_uk = df[df['location'] == 'United Kingdom'] #United Kingdom
data_uk_features = data_uk.drop(['location', 'date', 'new_cases'], axis = 1)
data_uk_target = data_uk['new_cases'].copy()

data_russia = df[df['location'] == 'Russia'] #Russia
data_russia_features = data_russia.drop(['location', 'date', 'new_cases'], axis = 1)
data_russia_target = data_russia['new_cases'].copy()

data_israel = df[df['location'] == 'Israel'] #Israel
data_israel_features = data_israel.drop(['location', 'date', 'new_cases'], axis = 1)
data_israel_target = data_israel['new_cases'].copy()

In [ ]:
scaler = StandardScaler()
data_ind_features_sc = scaler.fit_transform(data_ind_features)
data_usa_features_sc = scaler.fit_transform(data_usa_features)
data_uk_features_sc = scaler.fit_transform(data_uk_features)
data_russia_features_sc = scaler.fit_transform(data_russia_features)
data_israel_features_sc = scaler.fit_transform(data_israel_features)

In [ ]:
train_india_x, test_india_x, train_india_y, test_india_y  = train_test_split(data_ind_features_sc, data_ind_target, test_size = 0.2)
train_usa_x, test_usa_x , train_usa_y, test_usa_y = train_test_split(data_usa_features_sc, data_usa_target, test_size = 0.2)
train_uk_x, test_uk_x, train_uk_y, test_uk_y = train_test_split(data_uk_features_sc, data_uk_target, test_size = 0.2)
train_russia_x, test_russia_x, train_russia_y, test_russia_y = train_test_split(data_russia_features_sc, data_russia_target, test_size = 0.2)
train_israel_x, test_israel_x, train_israel_y, test_israel_y = train_test_split(data_israel_features_sc, data_israel_target, test_size = 0.2)

In [ ]:
india_lreg = LinearRegression()
usa_lreg = LinearRegression()
russia_lreg = LinearRegression()
uk_lreg = LinearRegression()
israel_lreg = LinearRegression()

In [ ]:
india_lreg.fit(train_india_x, train_india_y)
usa_lreg.fit(train_usa_x, train_usa_y)
russia_lreg.fit(train_russia_x, train_russia_y)
uk_lreg.fit(train_uk_x, train_uk_y)
israel_lreg.fit(train_israel_x, train_israel_y)

In [ ]:
def calcRMSE(model,x,y):
    predictions = model.predict(x)
    rmse = np.sqrt(mean_squared_error(y,predictions))
    return rmse

In [ ]:
rmse_for_lreg =list()

india_lreg_rmse = calcRMSE(india_lreg, test_india_x, test_india_y)
usa_lreg_rmse = calcRMSE(usa_lreg, test_usa_x, test_usa_y)
russia_lreg_rmse = calcRMSE(russia_lreg, test_russia_x, test_russia_y)
uk_lreg_rmse = calcRMSE(uk_lreg, test_uk_x, test_uk_y)
israel_lreg_rmse = calcRMSE(israel_lreg, test_israel_x, test_israel_y)

rmse_for_lreg.extend([india_lreg_rmse, usa_lreg_rmse, russia_lreg_rmse, israel_lreg_rmse])

print(rmse_for_lreg)

In [ ]:
india_rfr = RandomForestRegressor()
usa_rfr = RandomForestRegressor()
uk_rfr = RandomForestRegressor()
russia_rfr = RandomForestRegressor()
israel_rfr = RandomForestRegressor()

In [ ]:
india_rfr.fit(train_india_x, train_india_y)
usa_rfr.fit(train_usa_x, train_usa_y)
uk_rfr.fit(train_uk_x, train_uk_y)
russia_rfr.fit(train_russia_x, train_russia_y)
israel_rfr.fit(train_israel_x, train_israel_y)

In [ ]:
rmse_for_rfr = list()

india_rfr_rmse = calcRMSE(india_rfr, test_india_x, test_india_y)
usa_rfr_rmse = calcRMSE(usa_rfr, test_usa_x, test_usa_y)
uk_rfr_rmse = calcRMSE(uk_rfr, test_uk_x, test_uk_y)
russia_rfr_rmse = calcRMSE(russia_rfr, test_russia_x, test_russia_y)
israel_rfr_rmse = calcRMSE(israel_rfr, test_israel_x, test_israel_y)

rmse_for_rfr.extend([india_rfr_rmse, usa_rfr_rmse, russia_rfr_rmse, israel_rfr_rmse])
print(rmse_for_rfr)

In [ ]:
pred_india_lreg = india_lreg.predict(data_ind_features_sc)
pred_usa_lreg = usa_lreg.predict(data_usa_features_sc)
pred_uk_lreg = uk_lreg.predict(data_uk_features_sc)
pred_russia_lreg = russia_lreg.predict(data_russia_features_sc)
pred_israel_lreg = israel_lreg.predict(data_israel_features_sc)

In [ ]:
pred_india_rfr = india_rfr.predict(data_ind_features_sc)
pred_usa_rfr = usa_rfr.predict(data_usa_features_sc)
pred_uk_rfr = uk_rfr.predict(data_uk_features_sc)
pred_russia_rfr = russia_rfr.predict(data_russia_features_sc)
pred_israel_rfr = israel_rfr.predict(data_israel_features_sc)

In [ ]:
fig=plt.figure(figsize=(12,10))
fig, plt.plot(data_ind['date'],data_ind['new_cases'],color = 'red', linewidth = 3)
fig, plt.title('India', size = 10)
plt.show()
fig.savefig('E:/webapp/india_graph/india_graph.png', bbox_inches = "tight")

fig1=plt.figure(figsize=(12,10))
plt.plot(data_usa['date'], data_usa['new_cases'], color = 'blue', linewidth = 3)
plt.title('United States of America', size = 10)
plt.show()
fig1.savefig('E:/webapp/usa_graph/usa_graph.png', bbox_inches = "tight")

fig2=plt.figure(figsize=(12,10))
plt.plot(data_uk['date'], data_uk['new_cases'], color = 'g' , linewidth = 3)
plt.title('United Kingdom', size = 10)
plt.show()
fig2.savefig('E:/webapp/uk_graph/uk_graph.png', bbox_inches = "tight")

fig3=plt.figure(figsize=(12,10))
plt.plot(data_russia['date'], data_russia['new_cases'], color ='orange', linewidth = 3)
plt.title('Russia', size = 10)
plt.show()
fig3.savefig('E:/webapp/russia_graph/russia_graph.png', bbox_inches = "tight")

fig4=plt.figure(figsize=(12,10))
plt.plot(data_israel['date'], data_israel['new_cases'], color = 'cyan', linewidth = 3)
plt.title('Israel', size = 10)
plt.show()
fig4.savefig('E:/webapp/iran_graph/iran_graph.png', bbox_inches = "tight")

In [ ]:
fig=plt.figure(figsize=(12,10))
plt.plot(data_ind['date'],data_ind['new_cases'])
plt.plot(data_ind['date'],pred_india_lreg)
plt.legend(['New Cases','New Cases as predicted by Linear Regression'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.show()
fig.savefig("E:/webapp/india_lr_pred/india_lr_pred.png", bbox_inches = "tight")

fig1=plt.figure(figsize=(12,10))
plt.plot(data_usa['date'],data_usa['new_cases'])
plt.plot(data_usa['date'],pred_usa_lreg)
plt.legend(['New Cases','New Cases as predicted by Linear Regression'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.show()
fig1.savefig("E:/webapp/usa_lr_pred/usa_lr_pred.png", bbox_inches = "tight")

fig2=plt.figure(figsize=(12,10))
plt.plot(data_uk['date'],data_uk['new_cases'])
plt.plot(data_uk['date'],pred_uk_lreg)
plt.legend(['New Cases','New Cases as predicted by Linear Regression'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.show()
fig2.savefig("E:/webapp/uk_lr_pred/uk_lr_pred.png", bbox_inches = "tight")

fig3=plt.figure(figsize=(12,10))
plt.plot(data_russia['date'],data_russia['new_cases'])
plt.plot(data_russia['date'],pred_russia_lreg)
plt.legend(['New Cases','New Cases as predicted by Linear Regression'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.show()
fig3.savefig("E:/webapp/russia_lr_pred/russia_lr_pred.png", bbox_inches = "tight")

fig4=plt.figure(figsize=(12,10))
plt.plot(data_israel['date'],data_israel['new_cases'])
plt.plot(data_israel['date'],pred_israel_lreg)
plt.legend(['New Cases','New Cases as predicted by Linear Regression'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.show()
fig4.savefig("E:/webapp/iran_lr_pred/iran_lr_pred.png", bbox_inches = "tight")

In [ ]:
fig=plt.figure(figsize=(12,10))
plt.plot(data_ind['date'],data_ind['new_cases'])
plt.plot(data_ind['date'],pred_india_rfr, linewidth= 1)
plt.legend(['New Cases','New Cases as predicted by Random Forest Regressor'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.show()
fig.savefig("E:/webapp/india_rfr_pred/india_rfr_pred.png", bbox_inches = "tight")

fig1=plt.figure(figsize=(12,10))
plt.plot(data_usa['date'],data_usa['new_cases'])
plt.plot(data_usa['date'],pred_usa_rfr, linewidth= 1)
plt.legend(['New Cases','New Cases as predicted by Random Forest Regressor'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.show()
fig1.savefig("E:/webapp/usa_rfr_pred/usa_rfr_pred.png", bbox_inches = "tight")

fig2=plt.figure(figsize=(12,10))
plt.plot(data_uk['date'],data_uk['new_cases'])
plt.plot(data_uk['date'],pred_uk_rfr,linewidth= 1)
plt.legend(['New Cases','New Cases as predicted by Random Forest Regressor'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.show()
fig2.savefig("E:/webapp/uk_rfr_pred/uk_rfr_pred.png", bbox_inches = "tight")

fig3=plt.figure(figsize=(12,10))
plt.plot(data_russia['date'],data_russia['new_cases'])
plt.plot(data_russia['date'],pred_russia_rfr, linewidth= 1)
plt.legend(['New Cases','New Cases as predicted by Random Forest Regressor'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.show()
fig3.savefig("E:/webapp/russia_rfr_pred/russia_rfr_pred.png", bbox_inches = "tight")

fig4=plt.figure(figsize=(12,10))
plt.plot(data_israel['date'],data_israel['new_cases'])
plt.plot(data_israel['date'],pred_israel_rfr, linewidth= 1)
plt.legend(['New Cases','New Cases as predicted by Random Forest Regressor'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.show()
fig4.savefig("E:/webapp/iran_rfr_pred/iran_rfr_pred.png", bbox_inches = "tight")

In [ ]:
#ADD YTICK
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(countries1 , rmse_for_lreg, color = 'b', width = 0.25, align='edge' )

ax.bar(countries1 , rmse_for_rfr, color = 'g', width = -0.25, align='edge')
plt.legend(['Linear Regression','Random Forest Regressor'], prop={'size': 15})
plt.xlabel('Countries')
plt.ylabel('Root Mean Squared Error')
plt.title("RMSE Scores for Linear Regression and Random Forest Regressor Models")
fig.savefig('E:/webapp/rmse_for_lreg/rmse_for_lreg.png', bbox_inches = "tight")

In [ ]:
csv_data = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
dataframe = pd.read_csv(csv_data, usecols= ["location", "date", "new_cases"])
dataframe['date'] = pd.to_datetime(dataframe.date)
dataframe.dropna(inplace=True)
dataframe.tail()

In [ ]:
data_ind_arima = dataframe[dataframe['location'] == 'India']
data_usa_arima = dataframe[dataframe['location'] == 'United States']
data_uk_arima = dataframe[dataframe['location'] == 'United Kingdom']
data_russia_arima = dataframe[dataframe['location'] == 'Russia']
data_israel_arima = dataframe[dataframe['location'] == 'Israel']

# FORECAST AND ACTUAL VALUE GRAPH COMPARISON BEGINS HERE

In [ ]:
training_data_india = data_ind_arima[:-7]

training_data_usa = data_usa_arima[:-7]
training_data_uk = data_uk_arima[:-7]
training_data_russia = data_russia_arima[:-7]
training_data_israel = data_israel_arima[:-7]
training_data_india.tail()

In [ ]:
india_arima = ARIMA(training_data_india['new_cases'], order = (5,2,5))
india_arima_fit = india_arima.fit()

In [ ]:
usa_arima = ARIMA(training_data_usa['new_cases'], order = (5,1,4))
usa_arima_fit = usa_arima.fit()

In [ ]:
uk_arima = ARIMA(training_data_uk['new_cases'], order = (5,2,3))
uk_arima_fit = uk_arima.fit()

In [ ]:
russia_arima = ARIMA(training_data_russia['new_cases'], order = (5,2,5))
russia_arima_fit = russia_arima.fit()

In [ ]:
israel_arima = ARIMA(training_data_israel['new_cases'], order = (5,2,2))
israel_arima_fit = israel_arima.fit()

In [ ]:
india_forecasts = india_arima_fit.forecast(steps = 8)

usa_forecasts = usa_arima_fit.forecast(steps = 8)

uk_forecasts = uk_arima_fit.forecast(steps = 8)

russia_forecasts = russia_arima_fit.forecast(steps = 8)


israel_forecasts = israel_arima_fit.forecast(steps = 8)

In [ ]:
dates = list()
for i in range(1,9):
    date = date.today()
    increment = timedelta(days = i)
    date = date - increment
    dates.append(date)


dates.reverse()


In [ ]:
data_ind_check = data_ind_arima.tail(7)
data_usa_check = data_usa_arima.tail(7)
data_uk_check = data_uk_arima.tail(7)
data_russia_check = data_russia_arima.tail(7)
data_israel_check = data_israel_arima.tail(7)

In [ ]:
fig=plt.figure(figsize=(12,10))
plt.plot(data_ind_check['date'], data_ind_check['new_cases'])
plt.plot(dates, india_forecasts)
plt.legend(['New Cases','New Cases as predicted by ARIMA'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.grid()
plt.show()

fig=plt.figure(figsize=(12,10))
plt.plot(data_usa_check['date'], data_usa_check['new_cases'])
plt.plot(dates, usa_forecasts)
plt.legend(['New Cases','New Cases as predicted by ARIMA'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.grid()
plt.show()

fig=plt.figure(figsize=(12,10))
plt.plot(data_uk_check['date'], data_uk_check['new_cases'])
plt.plot(dates, uk_forecasts)
plt.legend(['New Cases','New Cases as predicted by ARIMA'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.grid()
plt.show()

fig=plt.figure(figsize=(12,10))
plt.plot(data_russia_check['date'], data_russia_check['new_cases'])
plt.plot(dates, russia_forecasts)
plt.legend(['New Cases','New Cases as predicted by ARIMA'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.grid()
plt.show()

fig=plt.figure(figsize=(12,10))
plt.plot(data_israel_check['date'], data_israel_check['new_cases'])
plt.plot(dates, israel_forecasts)
plt.legend(['New Cases','New Cases as predicted by ARIMA'], prop={'size': 12})
plt.xlabel('Date')
plt.ylabel('New Cases')
plt.grid()
plt.show()


In [ ]:
india_param = (5,2,5)
usa_param = (5,1,4)
uk_param = (5,2,3)
russia_param = (5,2,5)
israel_param = (5,2,2)

In [ ]:
def evaluateModel(data, param):
    X = data['new_cases'].values
    size = int(len(X) * 0.70)
    train, test = X[0:size], X[size:len(X)]
    history = [x for x in train]
    predictions = list()
    # walk-forward validation
    for t in range(len(test)):
        model = ARIMA(history, order=param, initialization='approximate_diffuse')
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
        print('predicted=%f, expected=%f' % (yhat, obs))
    rmse = np.sqrt(mean_squared_error(test, predictions))
    return rmse

In [ ]:
india_arima_rmse = evaluateModel(data_ind_arima, india_param) 
print(india_arima_rmse)

In [ ]:
usa_arima_rmse = evaluateModel(data_usa_arima, usa_param)
print(usa_arima_rmse)

In [ ]:
uk_arima_rmse = evaluateModel(data_uk_arima, uk_param)
print(uk_arima_rmse)

In [ ]:
russia_arima_rmse = evaluateModel(data_russia_arima, russia_param)
print(russia_arima_rmse)

In [ ]:
israel_arima_rmse = evaluateModel(data_israel_arima, israel_param)
print(israel_arima_rmse)

In [ ]:
rmse_for_arima = list()
rmse_for_arima.extend([india_arima_rmse, usa_arima_rmse, russia_arima_rmse, israel_arima_rmse])

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(countries1 , rmse_for_lreg, color = 'b', width = 0.25, align='edge' )

ax.bar(countries1 , rmse_for_rfr, color = 'g', width = -0.25, align='edge')
plt.legend(['Linear Regression','Random Forest Regressor'], prop={'size': 15})
plt.xlabel('Countries')
plt.ylabel('Root Mean Squared Error')
plt.title("RMSE Scores for Linear Regression and Random Forest Regressor Models")
fig.savefig('E:/webapp/rmse_for_rfr/rmse_for_rfr.png', bbox_inches = "tight")

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(countries1 , rmse_for_rfr, color = 'b', width = 0.25, align='edge' )

ax.bar(countries1 , rmse_for_arima, color = 'g', width = -0.25, align='edge')
plt.legend(['Random Forest Regressor', 'ARIMA Model'], prop={'size': 15})
plt.xlabel('Countries')
plt.ylabel('Root Mean Squared Error')
plt.title("RMSE Scores for Random Forest Regressor and ARIMA Models")
fig.savefig('E:/webapp/rmse_lreg_rfr/rmse_lreg_rfr.png', bbox_inches = "tight")

# END; NOW ACTUAL FORECASTING BEGINS

In [ ]:
india_arima = ARIMA(data_ind_arima['new_cases'], order = (5,2,5))
india_arima_fit = india_arima.fit()

In [ ]:
usa_arima = ARIMA(data_usa_arima['new_cases'], order = (5,1,4))
usa_arima_fit = usa_arima.fit()

In [ ]:
uk_arima = ARIMA(data_uk_arima['new_cases'], order = (5,2,3))
uk_arima_fit = uk_arima.fit()

In [ ]:
russia_arima = ARIMA(data_russia_arima['new_cases'], order = (5,2,5))
russia_arima_fit = russia_arima.fit()

In [ ]:
israel_arima = ARIMA(data_israel_arima['new_cases'], order = (5,2,2))
israel_arima_fit = israel_arima.fit()

In [ ]:
india_forecasts = india_arima_fit.forecast(steps = 7)

usa_forecasts = usa_arima_fit.forecast(steps = 7)
uk_forecasts = uk_arima_fit.forecast(steps = 7)
russia_forecasts = russia_arima_fit.forecast(steps = 7)
israel_forecasts = israel_arima_fit.forecast(steps = 7)

In [ ]:
dates = list()
for i in range(7):
    date = date.today()
    increment = timedelta(days = i)
    date = date + increment
    dates.append(date)
    
    
print(dates)

In [ ]:
forecastdict = {'Date': dates, 'India': india_forecasts}
forecastdata = pd.DataFrame(forecastdict, columns = ['Date', 'India'])

print(forecastdata)

In [ ]:
fig=plt.figure(figsize=(12,10))
plt.plot(dates, india_forecasts)
plt.grid()
plt.yticks(india_forecasts)
plt.show()
fig.savefig('E:/webapp/india_forecast/india_forecast.png', bbox_inches = "tight")

fig1=plt.figure(figsize=(12,10))
plt.plot(dates, usa_forecasts)
plt.grid()
plt.yticks(usa_forecasts)
plt.show()
fig1.savefig('E:/webapp/usa_forecast/usa_forecast.png', bbox_inches = "tight")

fig2=plt.figure(figsize=(12,10))
plt.plot(dates, uk_forecasts)
plt.grid()
plt.yticks(uk_forecasts)
plt.show()
fig2.savefig('E:/webapp/uk_forecast/uk_forecast.png', bbox_inches = "tight")

fig3=plt.figure(figsize=(12,10))
plt.plot(dates, russia_forecasts)
plt.grid()
plt.yticks(russia_forecasts)
plt.show()
fig3.savefig('E:/webapp/russia_forecast/russia_forecast.png', bbox_inches = "tight")

fig4=plt.figure(figsize=(12,10))
plt.plot(dates, israel_forecasts)
plt.grid()
plt.yticks(israel_forecasts)
plt.show()
fig4.savefig('E:/webapp/iran_forecast/iran_forecast.png', bbox_inches = "tight")